In [1]:
import tensorflow as tf
import numpy as np
import yaml
from abc import abstractmethod

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# module definitions
class TensorDict(dict):
    """
    Class that contains a dictionary of tensors
    """
    def last(self):
        return self.get(list(self.keys())[-1])
    
    def get_many(self, t_list):
        return {x: self.get(x) for x in t_list}

activation_dict = {"relu": tf.nn.relu, None: None, "None": None}

def create_conv_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.conv2d(inputs=tensor_in, 
                                   filters=spec['filters'],
                                   kernel_size=spec['kernel_size'],
                                   padding=spec['padding'],
                                   activation=activation_dict[spec['activation']],
                                   name=name)}

def create_maxpool_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.max_pooling2d(inputs=tensor_in,
                                          pool_size=spec['pool_size'],
                                          strides=spec['strides'],
                                          name=name)}

def create_dense_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    # creates an extra layer that must be flattened as well
    out = {}
    out[name+'_flatten'] = tf.reshape(tensor_in, [-1, np.prod(tensor_in.shape[1:4])])
    out[name] = tf.layers.dense(inputs=out[name+'_flatten'],
                                units=spec['units'],
                                activation=activation_dict[spec['activation']],
                                name=name)
    return out
                      
def create_dropout_layer(tensor_in, spec, name, mode=tf.estimator.ModeKeys.TRAIN):
    return {name: tf.layers.dropout(inputs=tensor_in, rate=spec['rate'], training = mode == tf.estimator.ModeKeys.TRAIN) }

l_dict = {'CONV':create_conv_layer, 'POOL':create_maxpool_layer, 'FC': create_dense_layer, 'DROP': create_dropout_layer}

def tensorboard_summaries(conv_layers=None, dense_layers=None):
    if conv_layers is not None:
        for cl_name, cl in conv_layers.items():
            shape = cl.shape.as_list()
            for f in range(shape[3]):
                tf.summary.image("{}/filter/{}".format(cl_name, f), tf.reshape(cl[:, :, :, f], [tf.shape(cl)[0], shape[1], shape[2], 1]))  
    if dense_layers is not None:
        for dl_name, dl in dense_layers.items():
           tf.summary.histogram("{}/weights".format(dl_name), dl) 

        
class confEstimator(tf.estimator.Estimator):
    """
    Class that allows building a certain neural-net based on a configuration YAML file. It must be overcharged.
    """
            
    def __init__(self, conf_file, model_dir):
        self.conf_file = conf_file
        with open(conf_file) as f:
            self.conf = yaml.load(f)
        super(confEstimator, self).__init__(self.conf_model_fn, model_dir=model_dir)
        
    @abstractmethod
    def conf_model_fn(self, features, labels, model):
        pass
    
    def _get_tensor_dict(self, data_in, mode):
        tensor_dict = TensorDict()  # will contain all the information on the layers and tensors
        num_channels = 1 if len(data_in.shape)==3 else data_in.shape[3]
        tensor_dict['input_layer'] = tf.reshape(data_in, [-1, data_in.shape[1], data_in.shape[2], num_channels]) 
        for l_name, l_def in self.conf['layers'].items():
            tensor_dict.update(l_dict[l_def['type']](tensor_dict.last(), l_def['specs'], l_name, mode))
        return tensor_dict
    
    def get_layers_of_type(self, l_type):
        return [l_name for l_name, l_def in self.conf['layers'].items() if l_def['type']==l_type]
    

class confRegressorEstimator(confEstimator):
    """
    Class that allows for a image classification based on a convolutional neural-net which architecture is on a YAML file
    """
    def conf_model_fn(self, features, labels, mode):

        # tensor_dict
        td =  self._get_tensor_dict(features, mode)
        
        # tensorboard summary for weights and variables in the layers   
        tensorboard_summaries(conv_layers=td.get_many(self.get_layers_of_type('CONV')), 
                             dense_layers=td.get_many(self.get_layers_of_type('FC')))

        # predictions
        predictions = {
            "predictions": td.last()  # return the last predicted image
        }

        if mode == tf.estimator.ModeKeys.PREDICT:
            # wrap predictions into a class and return EstimatorSpec object
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
        
        # minimize on cross-entropy
        loss = tf.losses.mean_squared_error(labels=labels, predictions=td.last())  # loss is a scalar tensor

        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer(learning_rate=self.conf['learning_rate'])
            train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)


        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["predictions"]),
            "mean_absolute_error": tf.metrics.mean_absolute_error(labels=labels, predictionns=predictions["predictions"]), 
            "RMS": tf.metrics.root_mean_squared_error(labels=labels, predictions=predictions["predictions"])
        }
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)



In [3]:
import datetime
import sys
from pathlib import Path
import os

sys.path.append("/jup/projects/mtgtools2/")  # ensure we find the MTG libraries
import pymtg.fci.esl.training_data as esl
import pymtg

d = datetime.datetime.now()
RETINAS_JSON = Path(pymtg.__file__).parent.parent / "test/FCI_ESL/pyesl/config/retinas.json"
ESL_PATH = Path("/mnt/data/FCI/ESL/nc/output/20170615T152646/ch123/")
FCI_PATH = Path("/mnt/data/FCI/ESL/nc/swaths/")
train_size = 100
test_size = 20

files = esl.prepare_training_data(FCI_PATH, ESL_PATH, 0, str(RETINAS_JSON))
train_batch = list(files["train"].batches("ch123", train_size, 20000, num_batches=1))
train_x =train_batch[0][0]
train_y = train_batch[0][1]

In [ ]:
test_batch = list(files["test"].batches("ch123", test_size, 20000, num_batches=1))

In [ ]:
test_x = test_batch[0][0]
test_y = test_batch[0][1]

In [ ]:
# create the estimator
esl_regressor = confRegressorEstimator('example_config.yml', 
                                       model_dir="/mnt/data/tensorboard/esl_conf_fc_model_{}".format(d.strftime("%Y%M%dT%H%m%S")))

# train
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=train_x, 
    y=train_y,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
esl_regressor.train(input_fn=train_input_fn, steps=200)

# evaluate
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=test_x,
    y=test_y,
    num_epochs=1, 
    shuffle=False)
eval_results = esl_regressor.evaluate(input_fn=eval_input_fn)
print(eval_results)].shape

In [6]:
train_y.shape

(100, 113)